In [10]:
import json
import re
import pandas as pd
import urllib
from urllib.request import Request
from bs4 import BeautifulSoup

In [11]:
import urllib.request
from bs4 import BeautifulSoup

url = "http://www.alltime-athletics.com/w_200ok.htm"

with urllib.request.urlopen(url) as resp:
    processed_page = BeautifulSoup(resp.read(), "html.parser")

pre_tag = processed_page.find("pre")
values_text = pre_tag.text

# Split the text into individual lines
lines = values_text.split("\n")

values_array = []

for line in lines:
    # Remove any leading/trailing whitespaces
    line = line.strip()
    if line:
        # Split the line into values based on the pattern you provided
        values = line.split()

        # Check if the 3rd index contains a valid wind category
        wind_category = values[2]
        if wind_category[0] in ['+', '-', '±'] and wind_category[1:].replace('.', '').isdigit():
            # Combine the strings after wind and before country code
            name = ' '.join(values[3:values.index(values[5])])

            # Replace the original values with the combined name
            values = values[:3] + [name] + values[5:]

        if len(values[4]) != 3:
            # Concatenate the 3rd index with the 2nd index
            values[3] = values[3] + ' ' + values[4]
            del values[4]
        if not values[4].isupper():
            values[3] = values[3] + ' ' + values[4]
            del values[4]

        # Delete the value in the 6th index
        if len(values) > 6:
                del values[6]

        values_array.append(values)


# Join values after 5th index but before the last index in each row
for row in values_array:
    if len(row) > 7:
        joined_values = ' '.join(row[6:-1])
        row[6] = joined_values
        del row[7:-1]



for row in values_array:
    print(row)

['1', '21.34', '+1.3', 'Florence Griffith-Joyner', 'USA', '21.12.59', 'Seoul', '29.09.1988']
['2', '21.45', '+0.6', 'Shericka Jackson', 'JAM', '16.07.94', 'Eugene', '21.07.2022']
['3', '21.53', '+0.8', 'Elaine Thompson-Herah', 'JAM', '28.06.92', 'Tokyo', '03.08.2021']
['4', '21.55', '±0.0', 'Shericka Jackson', 'JAM', '16.07.94', 'Kingston', '26.06.2022']
['5', '21.56', '+1.7', 'Florence Griffith-Joyner', 'USA', '21.12.59', 'Seoul', '29.09.1988']
['6', '21.61', '+1.3', 'Gabrielle Thomas', 'USA', '07.12.96', 'Eugene', '26.06.2021']
['7', '21.62A', '-0.6', 'Marion Jones', 'USA', '12.10.75', 'Johannesburg', '11.09.1998']
['8', '21.63', '+0.2', 'Dafne Schippers', 'NED', '15.06.92', 'Beijing', '28.08.2015']
['9', '21.64', '+0.8', 'Merlene Ottey', 'JAM', '10.05.60', 'Bruxelles', '13.09.1991']
['10', '21.66', '-1.0', 'Merlene Ottey', 'JAM', '10.05.60', 'Zürich', '15.08.1990']
['10', '21.66', '+0.2', 'Elaine Thompson-Herah', 'JAM', '28.06.92', 'Beijing', '28.08.2015']
['10', '21.66', '+0.3', 'E

In [12]:
frame = pd.DataFrame.from_dict(values_array)
frame

,0,1,2,3,4,5,6,7
0,1,21.34,+1.3,Florence Griffith-Joyner,USA,21.12.59,Seoul,29.09.1988
1,2,21.45,+0.6,Shericka Jackson,JAM,16.07.94,Eugene,21.07.2022
2,3,21.53,+0.8,Elaine Thompson-Herah,JAM,28.06.92,Tokyo,03.08.2021
3,4,21.55,±0.0,Shericka Jackson,JAM,16.07.94,Kingston,26.06.2022
4,5,21.56,+1.7,Florence Griffith-Joyner,USA,21.12.59,Seoul,29.09.1988
...,...,...,...,...,...,...,...,...
2314,2230,22.59,-0.2,Anthonique Strachan,BAH,22.08.93,Nassau,25.06.2022
2315,2230,22.59,-0.6,Brittany Brown,USA,18.04.95,Nassau,19.08.2022
2316,2230,22.59,+1.1,Bassant Hemida,EGY,28.09.96,Bellinzona,12.09.2022
2317,2230,22.59,+0.1,Kendall Ellis,USA,08.03.96,Eagle Rock,06.05.2023


In [13]:
import csv

filename = 'women_200m.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'Place',
        'Time',
        'Wind',
        'Athlete',
        'Country',
        'Date of Birth',
        'City',
        'Event Date'
    ]
    writer = csv.writer(csvfile)
    
    writer.writerow(fieldnames)  # Write the header row
    
    for row in values_array:
        # Check if there are multiple strings between the date and the weird number      
        writer.writerow(row)
    
print(f"Data saved to {filename} successfully.")

Data saved to women_200m.csv successfully.
